# RAG Tutorial for i2b2 Developers using CSV (No ChromaDB Required)

This notebook demonstrates how Retrieval-Augmented Generation (RAG) works using just a CSV file with embedded vectors and patient notes — no ChromaDB required. It supports both OpenAI and a local model (Ollama/Qwen2) and simulates the Patient Data Object (PDO) pattern from i2b2.

---


In [33]:
## 1. Patient Picker + Contextual Summary

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

# Load consolidated table
df = pd.read_csv("consolidated_table.csv")

# Extract patient metadata
df["patient_num"] = df["patient_num"].astype(str)
patient_selector = widgets.Dropdown(
    options=sorted(df["patient_num"].unique()),
    description="Patient #:",
    layout=widgets.Layout(width='40%')
)

# Define asthma vs non-asthma classification
no_asthma_patient_nums = {
    "1000000002", "1000000003", "1000000009", "1000000010", "1000000013",
    "1000000023", "1000000036", "1000000040", "1000000047", "1000000048",
    "1000000052", "1000000063", "1000000064", "1000000068", "1000000071",
    "1000000082", "1000000086", "1000000087", "1000000093", "1000000101",
    "1000000103", "1000000107"
}

def has_asthma(patient_num):
    return patient_num not in no_asthma_patient_nums

summary_output = widgets.Output()

@widgets.interact(patient=patient_selector)
def update_summary(patient):
    summary_output.clear_output()
    patient_rows = df[df["patient_num"] == patient]
    if patient_rows.empty:
        return

    doc_row = patient_rows.iloc[0]
    document_text = eval(doc_row["metadata"])['chroma:document']
    asthma_status = "✅ Asthma" if has_asthma(patient) else "❌ No Asthma"

    with summary_output:
        display(Markdown(f"### Patient {patient} - {asthma_status}"))
        display(Markdown(f"**Date:** {doc_row['visit_date']}"))
        display(widgets.Textarea(value=document_text[:1500], layout=widgets.Layout(width='100%', height='300px')))
widgets.Label("Note Type Used: Provider Notes")
update_summary(patient_selector.value)
display(summary_output)



interactive(children=(Dropdown(description='Patient #:', layout=Layout(width='40%'), options=('1000000001', '1…

Output()

In [9]:
df

,patient_num,timestamp,collection_name,doc_id,embedding_id,embedding,embedding_format,metadata
0,1000000001,2025-04-29 18:22:34,my_collection,doc_1000000001,embedding_1000000001,0xE61D6D3B128E183DFBBC443B24E4133C3C9CB03C1D05...,FLOAT32,"{""patient_num"": ""1000000001"", ""encounter_id"": ..."
1,1000000001,2025-04-29 18:22:48,my_collection,doc_1000000001,embedding_1000000001,0x6A549D3BD6E50C3D6333D13B59D0D13B9041DB3C1B9C...,FLOAT32,"{""patient_num"": ""1000000001"", ""encounter_id"": ..."
2,1000000001,2025-04-29 18:23:00,my_collection,doc_1000000001,embedding_1000000001,0x7655CB39739F2F3DA7FFF93B1EDF143CB1E5C03C7BC5...,FLOAT32,"{""patient_num"": ""1000000001"", ""encounter_id"": ..."
3,1000000001,2025-04-29 18:23:10,my_collection,doc_1000000001,embedding_1000000001,0x2B81FC3A372CFF3C05783C3BE1EE4D3C594DA13C8A6D...,FLOAT32,"{""patient_num"": ""1000000001"", ""encounter_id"": ..."
4,1000000001,2025-04-29 18:23:30,my_collection,doc_1000000001,embedding_1000000001,0x8EED3D3A77D7063DE97F913B2E56263C1395A33C175E...,FLOAT32,"{""patient_num"": ""1000000001"", ""encounter_id"": ..."
5,1000000001,2025-04-29 18:23:39,my_collection,doc_1000000001,embedding_1000000001,0xD4A1003AF47C253D79B7763B68AB233C52B3263C5631...,FLOAT32,"{""patient_num"": ""1000000001"", ""encounter_id"": ..."
6,1000000002,2025-04-29 18:23:49,my_collection,doc_1000000002,embedding_1000000002,0x1379113BFC80BB3C0A2ED93B7C68173C4D54863C5478...,FLOAT32,"{""patient_num"": ""1000000002"", ""encounter_id"": ..."
7,1000000002,2025-04-29 18:24:01,my_collection,doc_1000000002,embedding_1000000002,0x78D1AE3C6AACFE3C99F8E83B95647F3C4436053DD9A5...,FLOAT32,"{""patient_num"": ""1000000002"", ""encounter_id"": ..."
8,1000000002,2025-04-29 18:24:15,my_collection,doc_1000000002,embedding_1000000002,0xB67CA63C9AA51F3DD3AC033C1203903C52D4F23C805F...,FLOAT32,"{""patient_num"": ""1000000002"", ""encounter_id"": ..."
9,1000000002,2025-04-29 18:24:27,my_collection,doc_1000000002,embedding_1000000002,0xFEA89C3CA5EBEB3C07F8043CB16B2B3C2B49FD3C8937...,FLOAT32,"{""patient_num"": ""1000000002"", ""encounter_id"": ..."


In [29]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

load_dotenv()

# Set up Azure credentials and token provider
azure_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    azure_credential, "https://cognitiveservices.azure.com/.default"
)

# Initialize the AzureChatOpenAI model using environment variables
model = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_OPENAI_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=token_provider
)

# Initialize the AzureOpenAIEmbeddings model using environment variables
embedding_model = AzureOpenAIEmbeddings(
    model=os.getenv("AZURE_EMBEDDING_MODEL"),
    azure_deployment=os.getenv("AZURE_EMBEDDING_DEPLOYMENT"),
    api_version=os.getenv("AZURE_EMBEDDING_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_EMBEDDING_ENDPOINT"),
    azure_ad_token_provider=token_provider,
    timeout=None,  # never timeout
    max_retries=2,  # try again twice
)

In [22]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import struct, binascii

# Function to convert HEX embedding to list of floats
def hex_to_vector(hex_str):
    hex_clean = hex_str[2:] if hex_str.startswith("0x") else hex_str
    return list(struct.unpack(f'{len(hex_clean)//8}f', binascii.unhexlify(hex_clean)))


In [24]:
# Build document vectors
vectors = df["embedding"].apply(hex_to_vector).tolist()
X = np.array(vectors)

In [25]:

# Embed a query
query = "Does the patient have asthma and take fluticasone?"
query_embedding = embedding_model.embed_query(query)

In [27]:

# Compute cosine similarity
similarities = cosine_similarity([query_embedding], X)[0]
df["similarity"] = similarities

# Filter top results
top_k = df[df["similarity"] > 0.45].sort_values(by="similarity", ascending=False).head(10)

# Show top retrieved chunks
for idx, row in top_k.iterrows():
    note = eval(row["metadata"])['chroma:document']
    print(f"Patient #{row['patient_num']} -  - Similarity: {row['similarity']:.2f}")
    print(note[:500], "...\n")

Patient #1000000001 -  - Similarity: 0.56
This is a 23-year-old Black female who has been receiving care at our clinic for several months. She speaks English and presents today, March 4, 2008, for a follow-up appointment due to persistent asthma symptoms. The patient has been managing extrinsic asthma without status asthmaticus and recent episodes of dizziness and giddiness. Additionally, she has reported an irregular menstrual cycle and a history of unspecified gastritis and gastroduodenitis.

**Subjective:**
The patient reports ongoin ...

Patient #1000000001 -  - Similarity: 0.55
This is a 23-year-old Black female who has been receiving care at our clinic for several months. She speaks English and presents today, August 11, 2008, for a follow-up appointment due to persistent asthma symptoms. The patient has a primary diagnosis of asthma, unspecified without mention of status asthmaticus, and has also been managing extrinsic asthma without status asthmaticus. Additionally, she has re

In [28]:
rag_prompt = f"""
Context:

{top_k['metadata'].apply(lambda x: eval(x)['chroma:document'][:500]).str.cat(sep="\n\n")}

Question: Does the patient have asthma and are they taking fluticasone?
Answer:
"""
print(rag_prompt[:1000])


Context:

This is a 23-year-old Black female who has been receiving care at our clinic for several months. She speaks English and presents today, March 4, 2008, for a follow-up appointment due to persistent asthma symptoms. The patient has been managing extrinsic asthma without status asthmaticus and recent episodes of dizziness and giddiness. Additionally, she has reported an irregular menstrual cycle and a history of unspecified gastritis and gastroduodenitis.

**Subjective:**
The patient reports ongoin

This is a 23-year-old Black female who has been receiving care at our clinic for several months. She speaks English and presents today, August 11, 2008, for a follow-up appointment due to persistent asthma symptoms. The patient has a primary diagnosis of asthma, unspecified without mention of status asthmaticus, and has also been managing extrinsic asthma without status asthmaticus. Additionally, she has reported recurring shortness of breath, dyspnea, chest pain, dizziness, irregul

In [31]:
result = model.invoke(rag_prompt)
# result = ollama.invoke(rag_prompt)


print("\nResponse from LLM:\n")
print(result.content)


Response from LLM:

Yes, the patient has asthma, as indicated by multiple follow-up visits for asthma management and the primary diagnosis of asthma without status asthmaticus. There is no specific mention of the patient taking fluticasone in the provided context.
